In [24]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import requests

path = "/content/drive/MyDrive/france_disaster_2018_2024.xlsx"
df = pd.read_excel(path)

df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

numeric_cols = [
    "severity_index", "casualties", "economic_loss_usd",
    "response_time_hours", "aid_amount_usd",
    "response_efficiency_score", "recovery_days",
    "latitude", "longitude"
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
)
df = df.drop_duplicates()

# We merge days in month
df_monthly = (
    df.groupby(['year', 'month'])
      .agg({
          'severity_index': 'sum',
          'casualties': 'sum',
          'economic_loss_usd': 'sum',
          'response_time_hours': 'mean',
          'aid_amount_usd': 'sum',
          'response_efficiency_score': 'mean',
          'recovery_days': 'mean',
          'latitude': 'mean',
          'longitude': 'mean',
      })
      .reset_index()
)
# count distater instead of getting the disaster type
disaster_counts = df.groupby(['year', 'month']).size().reset_index(name='num_disasters')
df_monthly = df_monthly.merge(disaster_counts, on=['year', 'month'])

# DOWNLOAD THE EXCEL FILE
#df_monthly.to_excel("france_monthly_disaster_2018_2024.xlsx", index=False)
#from google.colab import files
#files.download("france_monthly_disaster_2018_2024.xlsx")

# We get the first 6 rows from the current year (including the header)
def get_temperature_table(year):
    url = f"https://www.infoclimat.fr/climatologie/annee/{year}/paris-montsouris/valeurs/07156.html"
    response = requests.get(url)
    response.raise_for_status()
    tables = pd.read_html(response.text, flavor="lxml")
    df = tables[0]
    df.columns = [str(c).strip() for c in df.columns]
    return df.iloc[:6]

# We clean the website data by removing non numerical value, except in the header
def tidy_temperature_table(df, year):
    df = df.rename(columns={df.columns[0]: "metric"})
    df_long = df.melt(
        id_vars="metric",
        var_name="month",
        value_name="value"
    )
    month_map = {
        "janv": 1, "fev": 2, "mars": 3, "avr": 4,
        "mai": 5, "juin": 6, "juil": 7, "août": 8,
        "sept": 9, "oct": 10, "nov": 11, "dec": 12
    }
    df_long["month"] = df_long["month"].str.extract(r"(\w+)\.?\s?").iloc[:,0].map(month_map)
    df_long["value"] = (
        df_long["value"]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .str.extract(r"(-?\d+\.?\d*)") # extract numeric
        .astype(float)
    )
    df_long["year"] = year
    df_tidy = df_long.pivot_table(
        index=["year", "month"],
        columns="metric",
        values="value"
    ).reset_index()
    return df_tidy

all_years = []

# We get data from common years
for year in range(2018, 2025):
    print("Scraping year:", year)
    table = get_temperature_table(year)
    tidy = tidy_temperature_table(table, year)
    all_years.append(tidy)

df_temp_all = pd.concat(all_years, ignore_index=True)

# DOWNLOAD THE EXCEL FILE
#df_temp_all.to_excel("france_monthly_temp_2018_2024.xlsx", index=False)
#from google.colab import files
#files.download("france_monthly_temp_2018_2024.xlsx")

# We merge both dataframes
last_cols = df_temp_all.iloc[:, -5:]
df_merged = pd.concat([df_monthly.reset_index(drop=True),
                       last_cols.reset_index(drop=True)], axis=1)

# DOWNLOAD THE EXCEL FILE
#df_merged.to_excel("merged_table.xlsx", index=False)
#from google.colab import files
#files.download("merged_table.xlsx")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Scraping year: 2018


/tmp/ipython-input-940725610.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text, flavor="lxml")


Scraping year: 2019


/tmp/ipython-input-940725610.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text, flavor="lxml")


Scraping year: 2020


...

/tmp/ipython-input-940725610.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text, flavor="lxml")


Scraping year: 2024


/tmp/ipython-input-940725610.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text, flavor="lxml")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>